# Introduction
This is the code of **QROS** in real device.

# Readme
1. Please run the following code blocks one by one, and finnally you will get the result in the final block.
2. If you want to change the dataset, please modify the next code block.

In [13]:
lstDatabase=[[1, '/dataset_evaluation_storage_0'], [3, '/dataset_evaluation_storage_1'], [2, '/dataset_evaluation_storage_2']]

#################
# lstDatabase[i] means: Read dataset_0 in following code
# Num_Qubit_Data means: Number of qubits required for storage determined by data type
#################
Num_Data_Qubit, curDb = lstDatabase[1]

print("Finished!")

Finished!


In [14]:
import os
import csv
import numpy as np
import pandas as pd

from qiskit import QuantumCircuit, Aer, assemble, QuantumRegister, ClassicalRegister
from math import pi
import math
import numpy as np
from qiskit.visualization import plot_bloch_multivector, plot_histogram
from qiskit.circuit.library.standard_gates import XGate, HGate, ZGate


from qiskit import IBMQ, transpile

from qiskit import QuantumCircuit, Aer, assemble, QuantumRegister, ClassicalRegister
from math import pi
import math
import numpy as np


from qiskit.visualization import plot_bloch_multivector, plot_histogram
from qiskit.circuit.library.standard_gates import XGate, HGate, ZGate
from qiskit import execute
import pandas as pd
from qiskit.circuit.library import MCMT


import pandas as pd

import itertools

pathName = os.getcwd() + curDb

numFiles = []

fileNames = os.listdir(pathName)
fileNames.reverse()
for fileName in fileNames:
    if fileName.endswith(".csv"):
        numFiles.append(fileName)


Database = {}
Database['tables'] = []
table_id = 0
num_table = 0
for i in numFiles:
    file = open(os.path.join(pathName, i), "rU")
    reader = csv.reader(file, delimiter=',')
    id = 0
    table_temp = []
    for row in reader:
        if id == 0:
            num_attributes = len(row)
        row_temp = [table_id] + [id] + list(map(int, row))
        table_temp.append(row_temp)
        id = id + 1
    tableHead = ["tableid"] + ["id"] + [str(i) for i in range(num_attributes)]
    indexHead = ["row_" + str(i) for i in range(id)] 
    relation = pd.DataFrame(table_temp, columns = tableHead, index = indexHead)

    Database['tables'].append( {
        "table_id" : table_id,
        "num_attributes" : num_attributes,
        "num_row" : id,
        "relation" : relation,
    })

    table_id = table_id + 1
    num_table = num_table + 1
Database["num_table"] = num_table

def dec_to_bin(x, n):
    return format(x, f"0{n}b")
def generate_truthtable(data, list_of_qubitnum):
    df_bin = pd.DataFrame()
    for col, n in zip(data.columns, list_of_qubitnum):
        df_bin[col] = data[col].apply(dec_to_bin, args=(n,))
    df_expanded = pd.DataFrame()
    
    index = 0
    for col, n in zip(df_bin.columns, list_of_qubitnum):
        df_split = df_bin[col].astype(str).str.split("", expand=True)
        df_split = df_split.drop([0,n + 1], axis=1)
        for i in range(1,n + 1):
            if index < len(df_bin.columns) - 1:
                new_col = f"key{index}_{n - i}"
            else:
                new_col = f"value{0}_{n - i}"
            df_expanded[new_col] = df_split[i]
        index += 1
    return df_expanded

import itertools

def get_combinations(arr, n):
    result = []
    lengths = [len(sublist) for sublist in arr]
    total = sum(lengths)

    if n > total:
        return result
    if n == total:
        result.append({i: arr[i] for i in range(len(arr))})
        return result
    for indices in itertools.combinations(range(total), n):
        dic = {}
        count = 0
        for i, sublist in enumerate(arr):
            lst = []
            for j, element in enumerate(sublist):
                if count in indices:
                    lst.append(element)
            count += 1
            dic[i] = lst
    result.append(dic)
    return result

import copy
def delete_zeros(all_keys, zero_bits):
    result = []
    length_zero_bits = 0
    for i in range(len(zero_bits)):
        length_zero_bits += len(zero_bits[i])
    for n in range(length_zero_bits + 1):
        tobe_deleteds = get_combinations(zero_bits, n)
        for tobe_deleted in tobe_deleteds:
            temp_list = copy.deepcopy(all_keys)
            for k, v in tobe_deleted.items():
                for loc in v:
                    temp_list[k].remove(loc)
            result.append(temp_list)
    return result

def get_keyid_bitloc(key):
    s = key.replace("key", "")
    return s.split("_")

def get_valueid_bitloc(key):
    s = key.replace("value", "")
    return s.split("_")

############################################
############################################
#注意！！！value只会有一个，value0！！
############################################
############################################
def generate_Cnot_circuit(truth_table):
    truth_table_key = truth_table.filter(regex='^key')
    truth_table_value = truth_table.filter(regex='^value')
    result = {}
    key_cols = [col for col in truth_table_key.columns if col.startswith("key")]
    value_cols = [col for col in truth_table_value.columns if col.startswith("value")]
    for i, value_col in enumerate(value_cols):
        value_num, bit_num_value = get_valueid_bitloc(value_col)
        value_num = int(value_num)
        bit_num_value = int(bit_num_value)
        value_df = truth_table_value[[value_col]]
        merged_df = pd.concat([truth_table_key, value_df], axis=1)
        merged_df = merged_df.astype(int)
        merged_df = merged_df[merged_df[value_col] == 1]
        result[bit_num_value] = []
        for index, row in merged_df.iterrows():
            zero_bits = []
            for key_col in key_cols:
                key_num, bit_num = get_keyid_bitloc(key_col)
                key_num = int(key_num)
                bit_num = int(bit_num)
                if key_num >= len(zero_bits):
                    zero_bits.append([])
                if row[key_col] == 0:
                    zero_bits[key_num].append(bit_num)
            all_keys = []
            for key_col in key_cols:
                key_num, bit_num = get_keyid_bitloc(key_col)
                key_num = int(key_num)
                bit_num = int(bit_num)
                if key_num >= len(all_keys):
                    all_keys.append([])
                all_keys[key_num].append(bit_num)
            new_keys = delete_zeros(all_keys, zero_bits)
            for new_key in new_keys:
                result[bit_num_value].append(new_key)
        from collections import Counter
        counter = Counter(tuple(map(tuple, keys)) for keys in result[bit_num_value])
        result[bit_num_value] = [list(map(list, keys)) for keys, count in counter.items() if count % 2 == 1]
    return result
            
            
            
            


def apply_cnot_circuuit(dist_cnot_circuit, qc, names_key_register, name_value_register):
    qc_registers = qc.qregs
    reg_for_subcircuit_key = []
    for name in names_key_register:
        for reg in qc_registers:
            if reg.name == name:
                reg_for_subcircuit_key.append(reg)
                break
    reg_for_subcircuit_value = []
    for reg in qc_registers:
        if reg.name == name_value_register[0]:
            reg_for_subcircuit_value.append(reg)
            break
    reg_for_subcircuit = reg_for_subcircuit_key + reg_for_subcircuit_value
    sub_circuit = QuantumCircuit(*reg_for_subcircuit)
    lst_starting_point = []
    point = 0
    for reg in reg_for_subcircuit:
        lst_starting_point.append(point)
        point = point + reg.size
    for k, v in dist_cnot_circuit.items():
        for lst_2d in v:
            control_bit = []
            index = 0
            for lst_1d in lst_2d:
                control_bit = control_bit + [x + lst_starting_point[index] for x in lst_1d]
                index += 1
            target_bit = []
            target_bit = [] + [k + lst_starting_point[-1]]
            if control_bit:
                sub_circuit.mct(control_bit, target_bit, None, mode="noancilla")
            else:
                sub_circuit.x(target_bit)
    return sub_circuit

def create_states(num_states, num_qubit):
    states_register = QuantumRegister(num_qubit, "states")
    ancilla_qubit = QuantumRegister(1, "ancilla_qubit")
    states_circuit = QuantumCircuit(states_register, ancilla_qubit)
    num_H = math.floor(math.log(num_states, 2))
    if num_H != 0:
        states_circuit.h(range(num_H))
    num_rest_states = int(num_states - math.pow(2, num_H))
    if num_rest_states == 0:
        return states_circuit
    
    
    
    data= pd.DataFrame({
        "key0":[i for i in range(num_rest_states)],
        "value0":[1 for i in range(num_rest_states)]
    })
    truth_table = generate_truthtable(data, [num_qubit, 1])

    dist_cnot_circuit = generate_Cnot_circuit(truth_table)
    sub_circuit = apply_cnot_circuuit(dist_cnot_circuit, states_circuit, ["states"], ["ancilla_qubit"])
    states_circuit.compose(sub_circuit, qubits = states_register[:] + ancilla_qubit[:], inplace=True) # 这是不带to_instruction()的
    
    from qiskit.circuit.library.standard_gates import HGate
    ch_gate = HGate().control(1)

    states_circuit.append(ch_gate, ancilla_qubit[:] + [states_register[num_H]]) # 不带to_instruction()    
    states_circuit.compose(sub_circuit, qubits= states_register[:] + ancilla_qubit[:], inplace=True) # 不带to_instruction()    
    
    data_new = pd.DataFrame({
        "key0" : [i for i in range(int(math.pow(2, num_H)), int(math.pow(2, num_H)) + num_rest_states)],
        "value0" : [1 for i in range(int(math.pow(2, num_H)), int(math.pow(2, num_H)) + num_rest_states)]
    })
    truth_table_new = generate_truthtable(data_new, [num_qubit, 1])
    dist_cnot_circuit_new = generate_Cnot_circuit(truth_table_new)
    sub_circuit_new = apply_cnot_circuuit(dist_cnot_circuit_new, states_circuit, ["states"], ["ancilla_qubit"])
    states_circuit.compose(sub_circuit_new, qubits = states_register[:] + ancilla_qubit[:], inplace=True)
    
    return states_circuit

def dict_table_quantum_registers(Database):
    dict_table_quantum_registers = {}
    tables_temp = Database["tables"]
    for i in range(len(tables_temp)):
        table_temp = tables_temp[i]
        num_row_temp = table_temp["num_row"]
        num_row_qubit = int(math.ceil(math.log(num_row_temp, 2)))
        if num_row_temp == 1:
            num_row_qubit = 1
        num_data_qubit = Num_Data_Qubit
        
        qr_id = QuantumRegister(num_row_qubit, "id" + str(i))

        qrs_data = []
        for j in range(table_temp["num_attributes"]):
            qr_data_temp = QuantumRegister(num_data_qubit, "data_" + str(i) + "_" + str(j))
            qrs_data.append(qr_data_temp)
        
        dict_table_quantum_registers["table" + str(i)] = [qr_id] + qrs_data
    
    return dict_table_quantum_registers

# dict_temp = dict_table_quantum_registers(Database)
# for k, v in dict_temp.items():
#     print(k)
#     print(v)

def preparation(Database):
    dict_registers = dict_table_quantum_registers(Database)
    qr_ancilla = QuantumRegister(1, "ancilla")
    lst_all_qreg = []
    for table, qregs in dict_registers.items():
        lst_all_qreg = lst_all_qreg + qregs
    lst_all_qreg.append(qr_ancilla)
    qc = QuantumCircuit(*lst_all_qreg)

    Qregs_dict = {}
    for qreg in lst_all_qreg:
        Qregs_dict[qreg.name] = qreg
    
    tables_temp = Database["tables"]
    for i in range(len(tables_temp)):
        table_temp = tables_temp[i]
        num_row_temp = table_temp["num_row"]
        sub_circuit = create_states(num_row_temp, Qregs_dict["id" + str(i)].size)
        qc.compose(sub_circuit, Qregs_dict["id" + str(i)][:] + Qregs_dict["ancilla"][:], inplace=True)
    
    tables_temp = Database["tables"]
    for i in range(len(tables_temp)):
        table_temp = tables_temp[i]
        num_row_temp = table_temp["num_row"]
        num_row_qubit = int(math.ceil(math.log(num_row_temp, 2)))
        relation_temp = table_temp["relation"]
        for j in range(table_temp["num_attributes"]):
            df_temp = relation_temp[ ['id', str(j)]]
            truth_table_temp = generate_truthtable(df_temp, [num_row_qubit, Num_Data_Qubit])
            dist_cnot_circuit_temp = generate_Cnot_circuit(truth_table_temp)
            sub_circuit = apply_cnot_circuuit(dist_cnot_circuit_temp, qc, names_key_register= ["id" + str(i)],name_value_register=["data_" + str(i) + "_" + str(j)])
            qc.compose(sub_circuit, Qregs_dict["id" + str(i)][:] + Qregs_dict["data_" + str(i) + "_" + str(j)][:], inplace = True)
    
    

    return qc

qc = preparation(Database)

cr = ClassicalRegister(qc.num_qubits - 1)
qc.add_register(cr)
qc.measure(range(qc.num_qubits - 1), range(qc.num_qubits - 1))






qc.draw()

/var/folders/f4/6bt5s2514m74026zjyvfxn7r0000gn/T/ipykernel_7473/3249596603.py:49: DeprecationWarning: 'U' mode is deprecated
  file = open(os.path.join(pathName, i), "rU")


┌───┐                                                             »
     id0_0: ┤ H ├───────────────■─────────■─────────■─────────■────■──────────»
            └───┘   ┌───┐       │         │         │         │    │          »
     id0_1: ────────┤ H ├───────■────■────┼────■────■─────────┼────┼────■─────»
                    └─┬─┘       │    │    │    │    │         │  ┌─┴─┐┌─┴─┐   »
data_0_0_0: ──────────┼─────────┼────┼────┼────┼────┼─────────┼──┤ X ├┤ X ├───»
                      │         │    │    │    │  ┌─┴─┐┌───┐┌─┴─┐└───┘└───┘┌─┐»
data_0_0_1: ──────────┼─────────┼────┼────┼────┼──┤ X ├┤ X ├┤ X ├──────────┤M├»
                      │         │    │  ┌─┴─┐┌─┴─┐└┬─┬┘└───┘└───┘          └╥┘»
data_0_0_2: ──────────┼─────────┼────┼──┤ X ├┤ X ├─┤M├──────────────────────╫─»
            ┌───┐     │         │    │  └───┘└───┘ └╥┘                      ║ »
     id1_0: ┤ H ├─────┼─────────┼────┼──────────────╫─────────■─────────■───╫─»
            └───┘     │         │    │       ┌───┐  ║         │         │   ║ »
     id1_1: ──────────┼─────────┼────┼───────┤ H ├──╫─────────■────■────┼───╫─»
                      │         │    │       └─┬─┘  ║         │    │    │   ║ »
data_1_0_0: ──────────┼─────────┼────┼─────────┼────╫─────────┼────┼────┼───╫─»
                 ┌─┐  │         │    │         │    ║         │    │    │   ║ »
data_1_0_1: ─────┤M├──┼─────────┼────┼─────────┼────╫─────────┼────┼────┼───╫─»
                 └╥┘  │         │    │         │    ║         │    │  ┌─┴─┐ ║ »
data_1_0_2: ──────╫───┼─────────┼────┼─────────┼────╫─────────┼────┼──┤ X ├─╫─»
                  ║   │         │    │         │    ║         │    │  └───┘ ║ »
data_1_1_0: ──────╫───┼─────────┼────┼─────────┼────╫─────────┼────┼────────╫─»
                  ║   │         │    │         │    ║         │    │        ║ »
data_1_1_1: ──────╫───┼─────────┼────┼─────────┼────╫─────────┼────┼────────╫─»
                  ║   │         │    │         │    ║         │    │        ║ »
data_1_1_2: ──────╫───┼─────────┼────┼─────────┼────╫─────────┼────┼────────╫─»
            ┌───┐ ║   │  ┌───┐┌─┴─┐┌─┴─┐┌───┐  │    ║  ┌───┐┌─┴─┐┌─┴─┐      ║ »
   ancilla: ┤ X ├─╫───■──┤ X ├┤ X ├┤ X ├┤ X ├──■────╫──┤ X ├┤ X ├┤ X ├──────╫─»
            └───┘ ║      └───┘└───┘└───┘└───┘       ║  └───┘└───┘└───┘      ║ »
     c2: 13/══════╩═════════════════════════════════╩═══════════════════════╩═»
                  8                                 4                       3 »
«                 ┌─┐                                                         »
«     id0_0: ─────┤M├─────────────────────────────────────────────────────────»
«                 └╥┘     ┌─┐                                                 »
«     id0_1: ──────╫──────┤M├─────────────────────────────────────────────────»
«                  ║      └╥┘┌─┐                                              »
«data_0_0_0: ──────╫───────╫─┤M├──────────────────────────────────────────────»
«                  ║       ║ └╥┘                                              »
«data_0_0_1: ──────╫───────╫──╫───────────────────────────────────────────────»
«                  ║       ║  ║                                               »
«data_0_0_2: ──────╫───────╫──╫───────────────────────────────────────────────»
«                  ║       ║  ║                                               »
«     id1_0: ──────╫───■───╫──╫───■────■────────────■─────────■────────────■──»
«                  ║   │   ║  ║   │    │            │         │       ┌─┐  │  »
«     id1_1: ──■───╫───■───╫──╫───┼────┼───────■────┼────■────■───────┤M├──┼──»
«              │   ║ ┌─┴─┐ ║  ║ ┌─┴─┐  │  ┌─┐  │    │    │    │       └╥┘  │  »
«data_1_0_0: ──┼───╫─┤ X ├─╫──╫─┤ X ├──┼──┤M├──┼────┼────┼────┼────────╫───┼──»
«              │   ║ └───┘ ║  ║ └───┘  │  └╥┘  │    │    │    │        ║   │  »
«data_1_0_1: ──┼───╫───────╫──╫────────┼───╫───┼────┼────┼────┼────────╫───┼──»
«            ┌─┴─┐ ║  ┌─┐  ║  ║        │   ║   │    │    │    │        ║   │  »
«data_1_0_2:

In [15]:
from qiskit import QuantumCircuit, transpile, assemble
from qiskit import IBMQ, Aer
from qiskit.providers.ibmq import least_busy
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.tools.monitor import job_monitor

import json


# 加载噪声模型
with open('noise_model.pkl', 'rb') as file:
    noise_model = pickle.load(file)
simulator = AerSimulator()
backend = AerSimulator()
transpiled_circuit = transpile(qc, backend, optimization_level=0)
job = execute(transpiled_circuit, simulator, noise_model=noise_model)
result = job.result()
counts = result.get_counts(transpiled_circuit)
counts_json = json.dumps(counts)



gate_counts = qc.count_ops()

dataQuantity = 0
for table in Database['tables']:
    dataQuantity += table["num_row"] * table["num_attributes"]
mct_counts = gate_counts.get('mcx', 0) + gate_counts.get('x', 0) + gate_counts.get('cx', 0) + gate_counts.get('ccx', 0)

circuitInfo = {}
circuitInfo['Database'] = pathName
circuitInfo['dataQuantity'] = dataQuantity
circuitInfo['Qubits'] = transpiled_circuit.num_qubits
circuitInfo['Depth'] = transpiled_circuit.depth()
circuitInfo['Gates'] = qc.size()
circuitInfo['MCTGates'] = mct_counts
circuitInfo["Backend"] = str(backend)

with open("circuitInfo_row_0.json", "w") as outfile:
    json.dump(circuitInfo, outfile)

import pickle
from qiskit import QuantumCircuit, execute
from qiskit.providers.aer import AerSimulator


# 将 JSON 字符串写入文件
with open('counts_row_0.json', 'w') as f:
    f.write(counts_json)



# 监视运行过程
job_monitor(job)

print("Finished!")

Job Status: job has successfully run
Finished!


In [16]:
def toStrQROS(Database):
    tables = Database["tables"]
    dividedStrQROS = []
    allDict = dict_table_quantum_registers(Database)
#     print(dictRegisters)
    indexTable = 0
    
    for table in tables:
        tableStrQROS = []
        relation_temp = table["relation"]
        numRegisters = 1 + table["num_attributes"]
        dictRegisters = allDict["table"+str(indexTable)]
        for index, row in relation_temp.iterrows():
            indexRegister = 0
            strQROS = ""
            for col in row.iloc[1:]:
                bin_temp = dec_to_bin(col, dictRegisters[indexRegister].size)
                strQROS = bin_temp + strQROS
                indexRegister += 1
            tableStrQROS.append(strQROS)
        indexTable += 1
        dividedStrQROS.append(tableStrQROS)
        
    resultStrQROS = []
    for combination in itertools.product(*dividedStrQROS[::-1]):
    # 将每个组合的字符串拼接在一起
        basicStateQROS = ''.join(combination)
#         print(basicStateQROS)
        resultStrQROS.append(basicStateQROS)
    
    return resultStrQROS
        
resultStrQROS = toStrQROS(Database)
# print(resultStrQROS)  

# if all(len(s) == len(resultStrQROS[0]) for s in resultStrQROS):
#     print("所有字符串都是一样长")
# else:
#     print("字符串的长度不一样")


def QROSRatio(Database):
    ## 先把Database转化成大量字符串，然后做对比
    databaseStr = toStrQROS(Database)
    Ratio=calculate_ratio(counts, databaseStr)
    return Ratio
    
def calculate_ratio(measurement_results, correct_results):
    total_correct_count = 0
    total_measurement_count = sum(measurement_results.values())

    for result in correct_results:
        total_correct_count += measurement_results.get(result, 0)

    ratio = total_correct_count / total_measurement_count
    return ratio

# # 示例数据
# measurement_results = {'00': 10, '01': 5, '10': 15, '11': 10}
# correct_results = ['00', '11']

# # 计算正确率
# ratio = calculate_ratio(counts, resultStrQROS)
# print(f'正确率为: {ratio:.2%}')

dataQuantity = 0
for table in Database['tables']:
    dataQuantity += table["num_row"] * table["num_attributes"]

gate_counts = qc.count_ops()

mct_counts = gate_counts.get('mcx', 0) + gate_counts.get('x', 0) + gate_counts.get('cx', 0) + gate_counts.get('ccx', 0)
# 将字典的键和列表转换为集合
set_counts = set(counts.keys())
set_correctresult = set(toStrQROS(Database))

# 计算集合的交集，即既在字典的键中又在列表中的元素
intersection = set_counts  & set_correctresult
    
# result_dict = {}
# result_dict['Database'] = Database
# result_dict['dataQuantity'] = dataQuantity
# result_dict['Qubits'] = transpiled_circuit.num_qubits
# result_dict['Depth'] = transpiled_circuit.depth()
# result_dict['Gates'] = qc.size()
# result_dict['MCTGates'] = mct_counts
# result_dict["Backend"] = str(backend)
result_dict = circuitInfo
result_dict["Valid Ratio"] = QROSRatio(Database)
result_dict["numDataQubits"] = Num_Data_Qubit
result_dict["integrityRatio"] = len(intersection)/len(toStrQROS(Database))
print(result_dict)

{'Database': '/Users/litadou/Desktop/myjupyter/quantumDatabase/upload/noisy_simulator/dataset_evaluation_storage_1', 'dataQuantity': 9, 'Qubits': 14, 'Depth': 26, 'Gates': 43, 'MCTGates': 26, 'Backend': 'aer_simulator', 'Valid Ratio': 0.0576171875, 'numDataQubits': 3, 'integrityRatio': 0.6666666666666666}
